In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.8/231.8 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 7.5 MB/s eta 0:00:00


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.4 MB/s eta 0:00:00


In [ ]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 33.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import re
import string
##from pymorphy3 import MorphAnalyzer
#from stop_words import get_stop_words
from transformers import AutoTokenizer, AutoModel
from torch import cuda
from tqdm import tqdm
import numpy as np
import torch.nn.functional as F
import torch
from xgboost import XGBClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import numpy as np
from transformers import AutoTokenizer, AutoModel
from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
#import optuna
np.random.seed(0)

In [ ]:
import emoji

In [ ]:
data_train = pd.read_csv('/content/drive/MyDrive/Znatno_personal/Отчетность ВКР/Артефакты/Parsing_avito/train.csv')

In [ ]:
data_test = pd.read_csv('/content/drive/MyDrive/Znatno_personal/Отчетность ВКР/Артефакты/Parsing_avito/data_test(sp).csv')

In [ ]:
data_train = data_train.drop(columns=['Unnamed: 0'])

In [ ]:
data_test = pd.read_csv('/content/drive/MyDrive/Znatno_personal/Отчетность ВКР/Артефакты/Parsing_avito/Model for benchmark/data/test_data.csv')

In [ ]:
data_test = data_test.drop(columns=['Unnamed: 0'])

In [ ]:
data_train

,message.text,Оценка
0,Сестринское дело (1991). Перевоспитание (2020)...,2
1,"Не указано, Опыт работы: 0,5 года, Компания: К...",2
2,"Коммуникабельный, стрессоустойчивый и пунктуал...",2
3,Парикмахер с опытом работы в салоне. Мне нрави...,2
4,"**Обязанности:** * Чистка лица (механическая, ...",2
...,...,...
3404,"Косметолог с 9-летним опытом, увлеченный своей...",3
3405,Косметолог с опытом работы 2.8 года. Имею высш...,6
3406,"Не указано,Стаж работы: 2.1,Компания: Здоровье...",5
3407,"По образованию фельдшер-лаборант, стаж работы ...",8


In [ ]:
data_train['Оценка'].value_counts()

,count
Оценка,
2,379
4,379
5,379
6,379
7,379
0,379
1,379
3,378
8,378


In [ ]:
X = data_train['message.text']
y = data_train['Оценка']
X_test = data_test['message.text']
y_test = data_test['Оценка']

#Предобработка

In [ ]:
stop_w = [ "и", "в", "во", "не", "на", "но",
    "что", "он", "как", "я", "с", "со",
    "а", "то", "все", "она", "так", "его",
    "только", "у"]

In [ ]:
def convert_emojis_to_words(text):

    # Convert emojis to words
    text = emoji.demojize(text, delimiters=(" ", " "))

    # Remove the : from the words and replace _ with space
    text = text.replace("_", " ")

    return text

In [ ]:
symbols_pattern = re.compile(pattern = "["
    "@_!#$%^&*()<>?/\|}{~√•—"
                       "]+", flags = re.UNICODE) #спецсимволы
# двойные пробелы
space_pattern = re.compile('\s+')
#exclude = set(string.punctuation) # удаление пунктуации
def clear_text(text):
    """ Функция удаления спецсимволов"""
    # удаление спецсимволов и emoji
    pre = symbols_pattern.sub(r'',text)
    pre = re.sub(r'http\S+', '', pre)
    pre = convert_emojis_to_words(pre)


    return space_pattern.sub(' ', pre)

In [ ]:
def preprocess_text(text):
    """ Финальная функция для обработки """
    # srip + lower + punctuation
    sentence = (
        ''.join([x for x in str(text).strip().lower()])
    )
    #sentence = (
        #' '.join([word for word in sentence.split() if word not in stop_w ])

    return clear_text(sentence)

In [ ]:
X = X.apply(preprocess_text)

In [ ]:
X_test = X_test.apply(preprocess_text)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state = 0)

In [ ]:
X_train

,message.text
1371,знакома с большим количеством офисных программ...
1362,"медицинское образование, опыт работы в космето..."
234,привет. меня зовут роман владимирович. у меня ...
2991,ответственный и внимательный к деталям космето...
1931,"ответственная, трудолюбивая, открыта к новым з..."
...,...
835,обязанности: процедуры технического обслужива...
3264,"ответственная, коммуникабельная. обязанности: ..."
1653,я косметолог-эстетист с опытом работы с 2019 г...
2607,"коммуникабельная, вежливая, дисциплинированная..."


# **TF-IDF + LogisticRegression**

In [ ]:
numeric_features = ["Стаж работы","Возраст"]
numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
)

categorical_features = ["Тип занятости", "Сфера деятельности", "Образование", "Пол"]
categorical_transformer = Pipeline(
    steps=[
        ("encoder", OneHotEncoder(handle_unknown="ignore")),
        ("selector", SelectPercentile(chi2, percentile=50)),
    ]
)
text_transformer = Pipeline(
    steps = [
        ("vectorizer", TfidfVectorizer())
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
        ("text", text_transformer,'user_text_info')
    ]
)

In [ ]:
clf = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", LogisticRegression(max_iter=2000))]
)


clf.fit(X_train, y_train)
print("Accuracy score: %.3f" % clf.score(X_test, y_test))

Accuracy score: 0.729


In [ ]:
pred = clf.predict(X_test)

In [ ]:
balanced_accuracy = balanced_accuracy_score(y_test, pred)
print("Balanced Accuracy:", balanced_accuracy)

Balanced Accuracy: 0.5738768162375588


In [ ]:
f1_ = f1_score(y_test, pred, average='weighted')
print("F1 score:", f1_)

F1 score: 0.7085000033261254


# **Multilingual-e5-large-instruct**

In [ ]:
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("intfloat/multilingual-e5-large-instruct")
model = AutoModel.from_pretrained("intfloat/multilingual-e5-large-instruct")
model.to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

XLMRobertaModel(
  (embeddings): XLMRobertaEmbeddings(
    (word_embeddings): Embedding(250002, 1024, padding_idx=1)
    (position_embeddings): Embedding(514, 1024, padding_idx=1)
    (token_type_embeddings): Embedding(1, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): XLMRobertaEncoder(
    (layer): ModuleList(
      (0-23): 24 x XLMRobertaLayer(
        (attention): XLMRobertaAttention(
          (self): XLMRobertaSdpaSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): XLMRobertaSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-05, eleme

In [ ]:
def embed_text(text):
    t = tokenizer(text, padding= True, truncation=True, max_length=512,  return_tensors='pt' )
    with torch.no_grad():
      model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

In [ ]:
x = np.zeros((len(X_train), 1024))
y =np.array(y_train)


In [ ]:
for ind, sent in enumerate(tqdm(X_train)):
   x[ind] = embed_text(sent)

100%|██████████| 3068/3068 [00:55<00:00, 55.31it/s]


In [ ]:
x_test = np.zeros((len(X_test), 1024))
y_test =np.array(y_test)

In [ ]:
for ind, sent in enumerate(tqdm(X_test)):
  x_test[ind] = embed_text(sent)

100%|██████████| 440/440 [00:07<00:00, 55.62it/s]


In [ ]:
x_val = np.zeros((len(X_val), 1024))
y_val =np.array(y_val)

In [ ]:
for ind, sent in enumerate(tqdm(X_val)):
  x_val[ind] = embed_text(sent)

100%|██████████| 341/341 [00:06<00:00, 53.17it/s]


## LogisticRegression

In [ ]:
logistic_reg = LogisticRegression(max_iter=2000)
logistic_reg.fit(x, y)

LogisticRegression(max_iter=2000)

In [ ]:
pred = logistic_reg.predict(x_test)

In [ ]:
accuracy = accuracy_score(y_test, pred)
print("Accuracy for Logistic Regression:", accuracy_score(y_test, pred))
print("F1 score for Logistic Regression:",f1_score(y_test, pred, average='weighted'))
print("Balanced accuracy for Logistic Regression:", balanced_accuracy_score(y_test,pred))

Accuracy for Logistic Regression: 0.6477272727272727
F1 score for Logistic Regression: 0.6448496218961066
Balanced accuracy for Logistic Regression: 0.6316341262835344


In [ ]:
X_val.to_list()

['обязанности ламинирование ресниц; косметические процедуры массаж лица, пилинг, чистка лица механическая, ультразвуковая, алмазная, комплексы ухода др. инъекционные процедуры биоревитализация, мезотерапия; применение косметических процедур умывания атравматичные механические, пилинги массажи; лазерная эпиляция лазер mediostar next pro mediostar monolith. компания «эпилас», парикмахерская. студия перманентного макияжа красоты. должность парикмахер, ламмейкер. учебное заведение самарский государственный медицинский университет куйбышевский медицинский институт им. д.и. ульянова, г. самара, лечебный факультет, специальность «фельдшер». стаж работы 2,66 года. образование специальность «среднее образование».',
 'уверенная себе, коммуникабельная, стрессоустойчивая, способна к самообучению. обязанности указано. компания указано. учебные заведения нет информации. стаж работы отсутствует. образование высшее.',
 'молодой человек, имеющий высшее медицинское образование специализирующийся космето

## XGB

In [ ]:
def objective(trial):
    params = {
        "objective": "multi:softmax",
        "num_class": 9,
        "eval_metric": "mlogloss",
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.0001, 0.3),
        "max_depth": trial.suggest_int("max_depth", 3, 100),
        "n_estimators": trial.suggest_int("n_estimators", 50, 2000),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1.0)
    }

    model = XGBClassifier(**params)
    model.fit(x, y)

    y_pred = model.predict(x_val)
    return accuracy_score(y_val, y_pred)

# Запуск оптимизации
study = optuna.create_study(direction="maximize")  # Ищем max accuracy
study.optimize(objective, n_trials=50)

# Лучшие параметры
print("Лучшие параметры:", study.best_params)

NameError: name 'optuna' is not defined

In [ ]:
xgb = XGBClassifier(objective="multi:softmax", num_class=9, eval_metric="mlogloss", learning_rate=0.07078025811321921, max_depth=46, n_estimators = 1284, subsample=0.5458605221291573, colsample_bytree= 0.15268221810604315)
xgb.fit(x, y)
# Предсказания и оценка
y_pred = xgb.predict(x_test)
print("Accuracy for XGB:", accuracy_score(y_test, y_pred))
print("F1 score for XGB:",f1_score(y_test, y_pred, average='weighted'))
print("Balanced accuracy for XGB:",balanced_accuracy_score(y_test,y_pred))

Accuracy for XGB: 0.7204545454545455
F1 score for XGB: 0.7190905562251805
Balanced accuracy for XGB: 0.703164784191851


# GigaChat

In [ ]:
import os
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel

In [ ]:
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
model1 = AutoModel.from_pretrained('ai-sage/Giga-Embeddings-instruct', trust_remote_code=True)

config.json:   0%|          | 0.00/3.43k [00:00<?, ?B/s]

configuration_gigarembed.py:   0%|          | 0.00/3.20k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai-sage/Giga-Embeddings-instruct:
- configuration_gigarembed.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_gigarembed.py:   0%|          | 0.00/18.8k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai-sage/Giga-Embeddings-instruct:
- modeling_gigarembed.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

model-00003-of-00003.safetensors:   0%|          | 0.00/271M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/47.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.02M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
model1.to(device)

GigarEmbedModel(
  (latent_attention_model): LatentAttentionModel(
    (cross_attend_blocks): ModuleList(
      (0-1): 2 x PreNorm()
    )
  )
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048, padding_idx=2)
    (layers): ModuleList(
      (0-26): 27 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=11008, bias=False)
          (up_proj): Linear(in_features=2048, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-06)
  

In [ ]:
from tqdm import tqdm

In [ ]:
def get_embeddings(text):
  task_name_to_instruct = {"example": "получая резюме, присвой ему метку класса от 0 до 8"}
  query_prefix = task_name_to_instruct["example"] + "\nрезюме: "
  queries = [text]
  query_embeddings = model1.encode(queries, instruction=query_prefix)
  query_embeddings = F.normalize(query_embeddings, p=2, dim=1)
  return query_embeddings[0].cpu().numpy()

In [ ]:
x1 = np.zeros((len(X_train), 2048))

In [ ]:
y1 = np.array(y_train)

In [ ]:
for ind, sent in enumerate(tqdm(X_train)):
   x1[ind] =  get_embeddings(sent)

100%|██████████| 3068/3068 [05:40<00:00,  9.02it/s]


In [ ]:
x1_test = np.zeros((len(X_test), 2048))

In [ ]:
y1_test = np.array(y_test)

In [ ]:
for ind, sent in enumerate(tqdm(X_test)):
   x1_test[ind] =  get_embeddings(sent)

100%|██████████| 440/440 [00:48<00:00,  9.03it/s]


In [ ]:
x1_val = np.zeros((len(X_val), 2048))
y1_val = np.array(y_val)

In [ ]:
for ind, sent in enumerate(tqdm(X_val)):
   x1_val[ind] =  get_embeddings(sent)

100%|██████████| 341/341 [00:39<00:00,  8.60it/s]


In [ ]:
logistic_reg = LogisticRegression(max_iter=2000)
logistic_reg.fit(x1, y1)

LogisticRegression(max_iter=2000)

In [ ]:
pred = logistic_reg.predict(x1_test)

In [ ]:
print("Accuracy for Logistic Regression:", accuracy_score(y1_test, pred))
print("F1 score for Logistic Regression:", f1_score(y1_test, pred, average='weighted'))
print("Balanced accuracy for Logistic Regression:",balanced_accuracy_score(y1_test,pred))

Accuracy for Logistic Regression: 0.6545454545454545
F1 score for Logistic Regression: 0.6546870300004031
Balanced accuracy for Logistic Regression: 0.6405942742622588


In [ ]:
def objective(trial):
    params = {
        "objective": "multi:softmax",
        "num_class": 9,
        "eval_metric": "mlogloss",
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.001, 0.3),
        "max_depth": trial.suggest_int("max_depth", 3, 100),
        "n_estimators": trial.suggest_int("n_estimators", 50, 2000),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1.0)
    }

    model = XGBClassifier(**params)
    model.fit(x1, y1)

    y1_pred = model.predict(x1_val)
    return accuracy_score(y1_val, y1_pred)

# Запуск оптимизации
study = optuna.create_study(direction="maximize")  # Ищем max accuracy
study.optimize(objective, n_trials=50)

# Лучшие параметры
print("Лучшие параметры:", study.best_params)

[I 2025-03-14 13:44:57,154] A new study created in memory with name: no-name-7c03ffe0-3f22-4840-b86c-8c09625ee97b
<ipython-input-156-89775a92e603>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 0.001, 0.3),
[I 2025-03-14 13:51:02,805] Trial 0 finished with value: 0.8797653958944281 and parameters: {'learning_rate': 0.014418637197041095, 'max_depth': 13, 'n_estimators': 401, 'subsample': 0.7702200699105977, 'colsample_bytree': 0.4152360311921744}. Best is trial 0 with value: 0.8797653958944281.
<ipython-input-156-89775a92e603>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_logun

Лучшие параметры: {'learning_rate': 0.08416526825199985, 'max_depth': 60, 'n_estimators': 526, 'subsample': 0.5329348943634309, 'colsample_bytree': 0.92995391106007}


In [ ]:
xgb = XGBClassifier(objective="multi:softmax", num_class=9, eval_metric="mlogloss", learning_rate=0.08416526825199985, max_depth=60, n_estimators = 526, subsample=0.5329348943634309, colsample_bytree= 0.92995391106007)
xgb.fit(x1, y1)
# Предсказания и оценка
y1_pred = xgb.predict(x1_test)
print("Accuracy for XGB:", accuracy_score(y1_test, y1_pred))
print("F1 score for XGB:",f1_score(y1_test, y1_pred, average='weighted'))
print("Balanced accuracy for XGB:",balanced_accuracy_score(y1_test,y1_pred))

Accuracy for XGB: 0.7181818181818181
F1 score for XGB: 0.7206505019393209
Balanced accuracy for XGB: 0.7052088977148517


In [ ]:
xgb1 = XGBClassifier(objective="multi:softmax", num_class=9, eval_metric="mlogloss", learning_rate=0.07078025811321921, max_depth=45, n_estimators = 700, subsample=0.5458605221291573, colsample_bytree= 0.15268221810604315)
xgb1.fit(x1, y1)
# Предсказания и оценка
y_pred2 = xgb1.predict(x1_test)
print("Accuracy for XGB:", accuracy_score(y1_test, y_pred2))
print("F1 score for XGB:",f1_score(y1_test, y_pred2, average='weighted'))
print("Balanced accuracy for XGB:",balanced_accuracy_score(y1_test,y_pred2))